# Ordinalni kalkulator

In [1]:
from functools import total_ordering
from collections import Counter
import copy

'''
Klasa koja reprezentira ordinale u Cantorovoj normalnoj formi.
'''

@total_ordering
class Ordinal():
    def __init__(self, arg):
        '''
        Konstruktor.
        '''
        if isinstance(arg, int):
            if arg >= 0:
                arg = {Ordinal.zero: arg}
            else:
                raise ValueError("Ordinal ne može biti negativan broj")
        summands = Counter(arg)
        self.summands = Counter()
        for exp, coeff in summands.items():
            if not isinstance(coeff, int) or coeff < 0:
                raise ValueError("Koeficijent ne može biti ne prirodni broj")
            
            if isinstance(exp, int):
                exp = Ordinal.fromnat(exp)
            if type(exp) is not Ordinal:
                raise ValueError("Eksponent ne može biti ne Ordinal")
                
            if coeff > 0:
                self.summands[exp] += coeff
            
        
    def omega():
        '''
        Konstruira osnovni ordinal omega.
        '''
        return Ordinal({1:1})
    
    def fromnat(n):
        '''
        Pretvara obični prirodan broj iz tipa int u tip Ordinal.
        '''
        return Ordinal({Ordinal.zero:n})
    
    def coefficient(self, k):
        """
        Vraća faktor od k-tog sumanda ordinala self.
        """
        return list(dict(sorted(self.summands.items(),reverse=True)).values())[k] 
    
    def exponent(self, k):
        """
        Vraća eksponent od k-tog sumanda ordinala self.
        """
        return list(dict(sorted(self.summands.items(),reverse=True)).keys())[k] 
    
    @property
    def is_number(self):
        '''
        Funkcija provjerava je li ordinal prirodni broj.
        '''
        if len(self.summands) == 1:
            if self == Ordinal(self.coefficient(0)):
                return True
               
        return False
    
    @property
    def is_null(self):
        '''
        Funkcija provjerava je li ordinal nula.
        '''
        if len(self.summands) == 0:
            return True
        return False
    
    @property
    def is_successor(self):
        '''
        Funkcija koja određuje je li ordinal tipa sljedbenik.
        '''
        return self.summands[Ordinal.zero] != 0

    
    def equals_one(self):
        '''
        Funkcija provjerava je li ordinal jednak jedinici.
        '''
        return self == Ordinal(1)
    
    def equals_omega(self):
        '''
        Funkcija provjerava je li ordinal jednak najmanjem beskonačnom ordinalu omega.
        '''
        return self == Ordinal.omega()

    @staticmethod
    def _make_string(exp,coef):
        '''
        Pretvara sumand u latex izraz. 
        '''
        s = ''
        if exp.is_null:
            return s + str(coef)
        if exp.is_number:
            exp = exp.coefficient(0)

        if exp == 0:
            s += str(coef)
        elif coef == 0:
            s += "0"
        else:
            s += "\omega"
            
            if type(exp) is int and exp > 1:
                s += '^' + str(exp)
            elif type(exp) is Ordinal:
                s += '^'
                if not exp.equals_omega():
                    s += '{('
                s += str(exp)
                if not exp.equals_omega():
                    s += ')}'
            if coef != 1:
                s += '*' + str(coef)
        return s
    
    def __str__(self):
        summands = [self._make_string(exp,self.summands[exp]) for exp in self.summands]
        return ' + '.join(summands)
    
    def __repr__(self):
        return str(self)
    
    def _repr_latex_(self):
        return r"$%s$" % str(self)
    
    def __eq__(self, other):
        if isinstance(other, int):
            other = Ordinal(other)
            
        if type(other) is Ordinal:
            return self.summands == other.summands
        else:
            raise TypeError(self._cmp_error_string % (type(self), type(other)))

    def __lt__(self, other):
        if isinstance(other, int):
            other = Ordinal(other)
            
        if type(other) is Ordinal:
            i = 0
            j = 0

            while i < len(self.summands) and j < len(other.summands):
                coef1 = self.coefficient(i)
                coef2 = other.coefficient(j)
                exp1 = self.exponent(i)
                exp2 = other.exponent(j)

                if exp1 > exp2:
                    return False
                elif exp2 > exp1:
                    return True
                else:
                    if coef1 > coef2:
                        return False
                    elif coef2 > coef1:
                        return True

                i += 1
                j += 1

            if i >= len(self.summands):
                if j >= len(other.summands):
                    return False
                return True
            else:
                return False
        else:
            raise TypeError(self._cmp_error_string % (type(self), type(other)))
    
    def __hash__(self):
        return hash((Ordinal, frozenset(self.summands.items())))
    
    def __add__(self, other):
        '''
        Računa sumu ordinala self + other zadržavajući u CNF.
        '''
        if type(other) is int:
            if other >= 0:
                other = Ordinal(other)
            else:
                raise ValueError("Ne mogu se dodati negativni brojevi")
        
        if type(other) is not Ordinal:
            raise TypeError(f"Krivi tip podataka za zbrajanje: {type(self)} i {type(other)}")
                
        if self.is_null:
            return other
        if other.is_null:
            return self
                
        i = 0
        j = 0
        
        exp1 = self.exponent(i)
        exp2 = other.exponent(j)
        
        
        result = Counter()
        
        while exp1 > exp2:
            coef1 = self.coefficient(i)
            result[exp1] = coef1
            i += 1
            
            if i >= len(self.summands):
                break

            exp1 = self.exponent(i)
            
        
        resultCoef = other.coefficient(j)
        if i < len(self.summands) and exp1 == exp2:
            resultCoef += self.coefficient(i)
        
        result[exp2] = resultCoef

        j += 1
        while j < len(other.summands):
            exp2 = other.exponent(j)
            result[exp2] = other.summands[exp2]
            j += 1
            
        return Ordinal(result)

    def __radd__(self, other):
        '''
        Računa sumu broja i ordinala zadržavajući u CNF.
        '''
        if type(other) is int:
            if other >= 0:
                return self
            else:
                raise ValueError("Ne mogu se dodati negativni brojevi")
        else:
            raise TypeError(f"Krivi tip podataka za zbrajanje: {type(self)} i {type(other)}")
            
    def __mul__(self, other):
        '''
        Računa produkt ordinala self * other zadržavajući u CNF.
        '''
        if type(other) is int:
            if other >= 0:
                other = Ordinal(other)
            else:
                raise ValueError("Ne mogu se množiti negativni brojevi")
                
        if type(other) is not Ordinal:
            raise TypeError(f"Krivi tip podataka za množenje: {type(self)} i {type(other)}")

        if self.is_null or other.is_null:
            return Ordinal.zero
        
        i = 0
        j = 0
        
        exp1 = self.exponent(i)
        exp2 = other.exponent(j)
        
        result = Counter()
        
        while exp2 > 0:
            resultExp = exp1 + exp2
            result[resultExp] += other.coefficient(j)
            j += 1
            
            if j >= len(other.summands):
                break
                
            exp2 = other.exponent(j)
        
        if j < len(other.summands):
            resultCoef = other.coefficient(j) * self.coefficient(i)
            result[exp1] += resultCoef
            
            i += 1
            while i < len(self.summands):
                exp1 = self.exponent(i)
                result[exp1] += self.summands[exp1]
                i += 1
            
        return Ordinal(result)

    def __rmul__(self, other):
        '''
        Računa umnožak broja i ordinala zadržavajući u CNF.
        '''
        if type(other) is int:
            if other > 0:
                return self
            elif other == 0:
                return Ordinal.zero
            else:
                raise ValueError("Ne mogu se množiti negativni brojevi")
        else:
            raise TypeError(f"Krivi tip podataka za množenje: {type(self)} i {type(other)}")
            
    def __pow__(self, other):
        '''
        Računa rezultat potenciranja ordinala self ^ other zadržavajući u CNF.
        '''
        if type(other) is int:
            if other >= 0:
                other = Ordinal(other)
            else:
                raise ValueError("Ne može se potencirati ordinal na negativni broj")
        
        if type(other) is not Ordinal:
            raise TypeError(f"Krivi tip podataka za potenciranje: {type(self)} i {type(other)}")
        
        natTerm = False #ima li other = w^a1*n1+...+w^ak*nk u cnf zapisu ak=0
        
        if other.is_null:
            return Ordinal(1)
        
        if other.is_number:
            if self.is_number:
                # self < omega, other < omega
                result = self.summands[Ordinal.zero] ** other.summands[Ordinal.zero]
            else:
                # self >= omega, other < omega
                coef2 = other.coefficient(0)
                
                if coef2 == 0:
                    return Ordinal(1)
                if coef2 == 1:
                    return self
                
                i = 0
                j = 0
                result = Counter()
                
                exp1 = self.exponent(0)
                coef1 = self.coefficient(0)
                
                tmp = exp1*(coef2-1)
                
                while True:
                    if exp1.is_null:
                        break
                        
                    tmpExp = tmp + exp1
                    result[tmpExp] = coef1
                    i += 1
                    
                    if i >= len(self.summands):
                        break
                        
                    exp1 = self.exponent(i)
                    coef1 = self.coefficient(i)
                    
                if exp1.is_null:
                    if coef1 > 0:
                        tmpCoef = self.coefficient(0) * coef1;

                        for j in range(1,coef2):
                            tmpExp =  self.exponent(0) * (coef2-j)
                            result[tmpExp] = tmpCoef
                            
                            i = 1
                            while i < len(self.summands)-1:
                                exp1 = self.exponent(i)
                                coef1 = self.coefficient(i)

                                tmpExp =  self.exponent(0) * (coef2-j-1)

                                if tmpExp.is_null:
                                    return Ordinal.zero

                                tmpExp = tmpExp + exp1

                                if tmpExp.is_null:
                                    return Ordinal.zero
                                result[tmpExp] = coef1
                                i += 1

                        result[Ordinal.zero] = coef1
                    
        else:
            if self.is_null:
                result = 0
            elif self.equals_one():
                result = 1
            elif self.is_number:
                # 1 < self < omega, other >= omega
                result = Counter()
                tmp = Counter()

                for i in range(len(other.summands)):
                        exp2 = other.exponent(i)
                        coef2 = other.coefficient(i)
                        if exp2.is_null == False:
                            if exp2.is_number:
                                    exp2 = Ordinal(exp2.coefficient(0) - 1)
                            tmp[exp2] = coef2;
                        else:
                            natTerm = True

                if natTerm == True:
                    resCoef = self.coefficient(0) ** coef2;
                else:
                    resCoef = 1;

                result[Ordinal(tmp)] = resCoef
            else:
                # self >= omega, other >= omega
                exp1 = self.exponent(0)
                
                result = Counter()
                tmp = Counter()

                for i in range(len(other.summands)):
                        exp2 = other.exponent(i)
                        coef2 = other.coefficient(i)
                        if exp2.is_null == False:
                            tmp[exp2] = coef2
                        else:
                            natTerm = True

                tmpExp = exp1*Ordinal(tmp)
                if tmpExp.is_null:
                    return Ordinal.zero
                result[tmpExp] = 1
                
                
                if natTerm == True:
                    tmp2 = self ** coef2
                    return Ordinal(result) * tmp2

        return Ordinal(result)
    
    def __rpow__(self, other):
        '''
        Računa potenciju broja na ordinal zadržavajući u CNF.
        '''
        if type(other) is int:
            if other >= 0:
                other = Ordinal(other)
                return other ** self
            else:
                raise ValueError("Ne može se potencirati negativni broj na ordinal")
        else:
            raise TypeError(f"Krivi tip podataka za potenciranje: {type(self)} i {type(other)}")
            
    
    def Isummation(alfa_k,beta):
        '''
        Funkcija računa sumu lambda izraza alfa_k koji predstavlja ordinal do granice beta = w*i1+i0.
        '''
        sum = Ordinal.zero
        
        for i in range(beta.summands[Ordinal(1)]):
            alfa_fp2 = alfa_k(w*i+2)
            sumExp = alfa_fp2.exponent(0)+1

            sum = sum + Ordinal({sumExp:1})
            
        tmp = w*beta.summands[Ordinal(1)]
        if beta.summands[Ordinal.zero] != 0:
            sum = sum + alfa_k(tmp+0)
            for i in range(1,beta.summands[Ordinal.zero]):
                sum = sum + alfa_k(tmp+i)
        
        return sum
        
    def Iproduct(alfa_k,beta):
        '''
        Funkcija računa umnožak lambda izraza alfa_k koji predstavlja ordinal do granice beta = w*i1+i0.
        '''
        prod = Ordinal(1)
        
        for i in range(beta.summands[Ordinal(1)]):
            alfa_fp2 = alfa_k(w*i+2)
            if alfa_fp2.is_number:
                prodExp = 1
            else:
                prodExp = alfa_fp2.exponent(0) * w

            prod = prod*Ordinal({prodExp:1})
            
        tmp = w*beta.summands[Ordinal(1)]
        if beta.summands[Ordinal.zero] != 0:
            prod = prod*alfa_k(tmp+0)
            for i in range(1,beta.summands[Ordinal.zero]):
                prod = prod*alfa_k(tmp+i)
        
        return prod


Ordinal.zero = Ordinal({})
w = Ordinal.omega()

## Testiranje

In [2]:
w = Ordinal.omega()
r = Ordinal({w:3,1:1})
t = Ordinal(3)
o = Ordinal({w:3,1:1})
p = Ordinal({o:1,0:1})


assert o.is_successor == False
assert p.is_successor == True

assert (p == o) == False
assert (o < p) == True
assert (w < p) == True
assert (w > p) == False
assert (p == w) == False
assert (w > 1) == True

q = Ordinal({0:3})
assert q.is_number == True
assert q > 2

test1add = Ordinal({Ordinal({Ordinal({1:1}):3,1:1}):1,0:1})
assert o + p == test1add

test2add = Ordinal({Ordinal({Ordinal({1:1}):3,1:1}):1,Ordinal({1:1}):3,1:1})
assert p + o == test2add

test3sum = Ordinal({o:1,0:4})
assert p + 3 == test3sum

test1mul = Ordinal({w:6,1:1})
assert o * 2 == test1mul

test2mul = Ordinal({Ordinal({Ordinal({1:1}):3,1:2}):3,Ordinal({Ordinal({1:1}):3,1:1,0:1}):1})
assert p * o == test2mul

test1exp = Ordinal({Ordinal({1:1,0:1}):1, w:1})
assert (w+1) ** (w+1) == test1exp

test2exp = Ordinal({Ordinal({w:3,0:1}):1})
assert 2 ** o == test2exp

test3exp = Ordinal({Ordinal({w:3,2:1}):1})
assert o ** o == test3exp

In [3]:
Ordinal.zero.is_successor

False

In [4]:
w = Ordinal.omega()
w

\omega

In [5]:
r = Ordinal({w:3,1:1})
r

\omega^\omega*3 + \omega

In [6]:
t = Ordinal(3)
t

3

In [7]:
t.is_number

True

In [8]:
t.equals_one()

False

In [9]:
r.equals_omega()

False

In [10]:
o = Ordinal({w:3,1:1})
o

\omega^\omega*3 + \omega

In [11]:
p = Ordinal({o:1,0:1})
p

\omega^{(\omega^\omega*3 + \omega)} + 1

In [12]:
o + p

\omega^{(\omega^\omega*3 + \omega)} + 1

In [13]:
p + o

\omega^{(\omega^\omega*3 + \omega)} + \omega^\omega*3 + \omega

In [14]:
p + w

\omega^{(\omega^\omega*3 + \omega)} + \omega

In [15]:
p + 3

\omega^{(\omega^\omega*3 + \omega)} + 4

In [16]:
p

\omega^{(\omega^\omega*3 + \omega)} + 1

In [17]:
p + w + w + 1

\omega^{(\omega^\omega*3 + \omega)} + \omega*2 + 1

In [18]:
p * o

\omega^{(\omega^\omega*3 + \omega*2)}*3 + \omega^{(\omega^\omega*3 + \omega + 1)}

In [19]:
o * 2

\omega^\omega*6 + \omega

In [20]:
(w+1)**(w+1)

\omega^{(\omega + 1)} + \omega^\omega

In [21]:
w**w**w**w

\omega^{(\omega^{(\omega^\omega)})}

In [22]:
2 ** w ** 2

\omega^\omega

In [23]:
2 ** o

\omega^{(\omega^\omega*3 + 1)}

In [24]:
o ** (o+3)

\omega^{(\omega^\omega*3 + \omega^2 + \omega*3)}*3 + \omega^{(\omega^\omega*3 + \omega^2 + \omega*2 + 1)}

In [25]:
(w+3)**3

\omega^3 + \omega^2*3 + \omega*3 + 3

In [26]:
Ordinal.Isummation(lambda j: (w+j)**j, Ordinal(4))

\omega^3 + \omega^2*3 + \omega*3 + 3

In [27]:
Ordinal.Iproduct(lambda j: (w+j)**j, Ordinal(3))

\omega^3 + \omega^2*2 + \omega*2 + 1

In [28]:
######################## Iz zbirke ######################

In [29]:
#zadatak 190.
Ordinal.Isummation(lambda j: j*w+w*j, w+3)

\omega^2*7 + \omega*2

In [30]:
#zadatak 191.
Ordinal.Isummation(lambda j: j, w*2)

\omega^2

In [31]:
#zadatak 192.
Ordinal.Isummation(lambda j: (w+j)*w, w*2+3)

\omega^3*2 + \omega^2*3

In [32]:
#zadatak 193.
Ordinal.Isummation(lambda j: j**2, w)

\omega

In [33]:
#zadatak 195.a
Ordinal.Isummation(lambda j: 2+j+2**j, w*2+2)

\omega^2*4

In [34]:
#zadatak 195.b
Ordinal.Isummation(lambda i: i**3+3**i, w*3)

\omega^4*2

In [35]:
#zadatak 195.c
Ordinal.Isummation(lambda i: i + w*2, w*2)

\omega^2*2

In [36]:
#zadatak 195.e
Ordinal.Isummation(lambda i: i ** 2, w*2)

\omega^3

In [37]:
#zadatak 195.f
Ordinal.Isummation(lambda i: (w + i) * w ** (i + 2), w + 3)

\omega^{(\omega + 4)}

In [38]:
#zadatak 195.g
Ordinal.Isummation(lambda i: w + 2*i, w*2)

\omega^2*2

In [39]:
#zadatak 195.h
Ordinal.Isummation(lambda i: (i + 2) ** (w + i), w + 1)

\omega^{(\omega*2)}

In [40]:
#zadatak 195.i
Ordinal.Isummation(lambda j: j**w, w+2)

\omega^\omega*2

In [41]:
#zadatak 195.j
Ordinal.Isummation(lambda j: w, w*2)

\omega^2*2

In [42]:
#zadatak 195.m
Ordinal.Isummation(lambda j: w**j, w+2)

\omega^{(\omega + 1)}

In [43]:
#zadatak 199.
Ordinal.Iproduct(lambda j: j + w, w*3 + 2)

\omega^{(\omega*3 + 2)}*4

In [44]:
#zadatak 200.
Ordinal.Iproduct(lambda j: (j + 2) ** (j + 1), w + 2)

\omega^{(\omega*2 + 2)} + \omega^{(\omega*2 + 1)}*3 + \omega^{(\omega*2)}*3

In [45]:
#zadatak 201.
Ordinal.Iproduct(lambda j: w ** (w + j), w)

\omega^{(\omega^2)}

In [46]:
#zadatak 202.a
Ordinal.Iproduct(lambda i: w ** 2 + 2 ** i, w + 2)

\omega^{(\omega + 4)} + \omega^{(\omega + 3)}*2

In [47]:
#zadatak 202.b
Ordinal.Iproduct(lambda i: (2*i + w) ** 2, w + 3)

\omega^{(\omega + 6)}*2

In [48]:
#zadatak 202.c
Ordinal.Iproduct(lambda i: i + w*2 + i, w + 2)

\omega^{(\omega + 2)}*4 + \omega^{(\omega + 1)}*4

In [49]:

Ordinal.Iproduct(lambda i: 2 ** i + w, w*2)

\omega^{(\omega*2)}

In [50]:
#zadatak 209.a
3 ** Ordinal.Isummation(lambda i: 3**i, w*3)

\omega^{(\omega^2)}

In [51]:
#zadatak 209.b
2*w*(w+1)*(w+2)

\omega^3 + \omega^2*2 + \omega

In [52]:
(w+1) ** w

\omega^\omega